Objective: Load the data, standardize column names to prevent future errors, handle sensitive information, and perform initial cleaning.

Key Libraries: pandas, numpy

In [10]:
import pandas as pd
import numpy as np

# Construct the file path relative to the notebook's location
file_path = '../data/raw/DataCoSupplyChainDataset.csv'

# Load the dataset using 'latin1' encoding
try:
    df_raw = pd.read_csv(file_path, encoding='latin1')
    print("Dataset loaded successfully.")
    print(f"Shape of the dataset: {df.shape[0]} rows and {df.shape[1]} columns.")
except FileNotFoundError:
    print(f"Error: The file was not found at {file_path}")

Dataset loaded successfully.
Shape of the dataset: 180519 rows and 53 columns.


In [11]:
# --- FIX COLUMN NAMES ---
# Create a copy to keep the original raw data untouched
df_cleaned = df_raw.copy()

# Standardize column names
df_cleaned.columns = df_cleaned.columns.str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

# Print the new, easy-to-use column names
print("Standardized Column Names:")
print(df_cleaned.columns)

Standardized Column Names:
Index(['ï»¿type', 'days_for_shipping_real', 'days_for_shipment_scheduled',
       'benefit_per_order', 'sales_per_customer', 'delivery_status',
       'late_delivery_risk', 'category_id', 'category_name', 'customer_city',
       'customer_country', 'customer_email', 'customer_fname', 'customer_id',
       'customer_lname', 'customer_password', 'customer_segment',
       'customer_state', 'customer_street', 'customer_zipcode',
       'department_id', 'department_name', 'latitude', 'longitude', 'market',
       'order_city', 'order_country', 'order_customer_id',
       'order_date_dateorders', 'order_id', 'order_item_cardprod_id',
       'order_item_discount', 'order_item_discount_rate', 'order_item_id',
       'order_item_product_price', 'order_item_profit_ratio',
       'order_item_quantity', 'sales', 'order_item_total',
       'order_profit_per_order', 'order_region', 'order_state', 'order_status',
       'order_zipcode', 'product_card_id', 'product_category

In [12]:
# Display the first 5 rows to get a feel for the data
print("First 5 rows of the dataset:")
display(df.head())

# Get a concise summary of the DataFrame (data types, non-null values)
print("\nDataFrame Info:")
df.info()

First 5 rows of the dataset:


,ï»¿Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class



DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 53 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   ï»¿Type                        180519 non-null  object 
 1   Days for shipping (real)       180519 non-null  int64  
 2   Days for shipment (scheduled)  180519 non-null  int64  
 3   Benefit per order              180519 non-null  float64
 4   Sales per customer             180519 non-null  float64
 5   Delivery Status                180519 non-null  object 
 6   Late_delivery_risk             180519 non-null  int64  
 7   Category Id                    180519 non-null  int64  
 8   Category Name                  180519 non-null  object 
 9   Customer City                  180519 non-null  object 
 10  Customer Country               180519 non-null  object 
 11  Customer Email                 180519 non-null  object 
 12  Customer Fnam

In [13]:
# 3. Drop PII columns

pii_columns = [
    'customer_email', 'customer_fname', 'customer_lname', 
    'customer_password', 'customer_street'
]
df_cleaned = df_cleaned.drop(columns=pii_columns, errors='ignore')

In [14]:
# Convert 'order_date_dateorders' to datetime objects
df_cleaned['order_date_dateorders'] = pd.to_datetime(df_cleaned['order_date_dateorders'])

# Handle missing 'customer_zipcode' (e.g., fill with 0 or a placeholder)
df_cleaned['customer_zipcode'] = df_cleaned['customer_zipcode'].fillna(0)

# The 'product_description' column is almost entirely empty, so we'll drop it
df_cleaned = df_cleaned.drop(columns=['product_description'], errors='ignore')

print("\nConverted date columns and handled missing values.")


Converted date columns and handled missing values.


In [15]:
# Calculate and display the count of missing values for each column
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)

if not missing_values.empty:
    print("Columns with missing values:")
    print(missing_values)
else:
    print("No missing values found in the dataset.")

Columns with missing values:
Product Description    180519
Order Zipcode          155679
Customer Lname              8
Customer Zipcode            3
dtype: int64


In [16]:
# Save the fully cleaned data for the next step
df_cleaned.to_csv('../data/processed/cleaned_orders.csv', index=False)

print("\nCleaned data saved successfully to 'data/processed/cleaned_orders.csv'")


Cleaned data saved successfully to 'data/processed/cleaned_orders.csv'
